# Классификация данных Speedtest
В рамках данного ноутбука поработаем непосредственно с обработанными данными.

## Подготовка к работе
Оптимизируем дальнейший код и сразу заполним пространство всем необходимым функционалом.

In [1]:
import pandas as pd
import tensorflow as tf

import seaborn as sns

2023-05-12 09:18:31.892728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 09:18:33.175496: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
SEED = 0  # to reproduce building
BATCH_SIZE = 44

FIG_SIZE = (38.4,21.6)  # 3810x2160

DATA_PATH = 'data/merged'
LOGS_PATH = 'logs/merged'
VISUAL_PATH = 'visual/model/merged'
MODEL_PATH = 'model/merged'

In [3]:
sns.set_theme(palette='winter', font='jost', font_scale=3, rc={'figure.figsize': FIG_SIZE})

### Подготовка датасета
Загрузим датасет, отсортируем значения по годам и уровню ВВП и уберём лишние признаки

In [4]:
df = (
    pd.read_csv(f'{DATA_PATH}/fixed/fixed_merged.csv')
    .sort_values(by=['year', 'income'])
    .drop(columns=['iso_a3', 'label', 'income'])
)

df.head()

,year,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,group
39923564,2019,18136.0,16625.0,14.0,1,1,Low income
39923565,2019,35595.0,7792.0,686.0,1,1,Low income
39923566,2019,38425.0,1888.0,745.0,1,1,Low income
39923567,2019,764.0,502.0,108.0,1,1,Low income
39923568,2019,26352.5,6077.0,689.5,8,4,Low income


Закодируем исследуемую категорию

In [5]:
values, uniques = pd.factorize(df['group'])
df['group'] = values

print('\n'.join(f'{n} - {v}' for n, v in enumerate(uniques)))

0 - Low income
1 - Lower-middle income
2 - Upper-middle income
3 - High income


Разделим датасет на обучающий набор - 60%, и на тестовый - 40%

In [6]:
train_df = df.sample(frac=0.6, random_state=SEED)
test_df = df.drop(train_df.index)

print(len(train_df), 'training examples')
print(len(test_df), 'testing examples')

24595006 training examples
16396670 testing examples


Дропнем из признаков зависимую переменную, которую затем приведём к матричному виду (target_values, target_count)

In [7]:
train_features = train_df.copy()
train_target = tf.keras.utils.to_categorical(train_features.pop('group'))

In [8]:
test_features = test_df.copy()
test_target = tf.keras.utils.to_categorical(test_features.pop('group'))

Из ноутбука с визуализацией было выяснено, что признаки выраженное левостороннее распределение.
Создадим слой для их нормализации и подгоним его состояние предварительной обработки к признакам

In [ ]:
normalize = tf.keras.layers.Normalization()
normalize.adapt(train_features, batch_size=BATCH_SIZE // 4)

2023-05-12 09:19:35.178792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-12 09:19:35.407062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-12 09:19:35.407124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-12 09:19:35.409218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-12 09:19:35.409275: I tensorflow/compile

## Регрессия с глубокой нейронной сетью (DNN)
Регрессия с использованием DNN и нескольких входных данных

### Настройка модели
Зададим параметры обучения модели

Модель тем качественнее обучается, чем больше у неё есть время.
Оптимизируем обучение, замедляя его с каждой эпохой

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=1000,
    decay_rate=1,
)

В дальнейшем предполагается построение нескольких моделей, различающихся по крупности.
Напишем общую функцию их настройки и обучения:
* optimizer - формула, по которой происходит оптимизация модели
* loss - формула, по которой рассчитываются ошибки предсказаний модели
* batch_size - размер совокупности наблюдений, по которым модель обучается за раз
* callbacks - дополнительные параметры, в данном случае отвечающие за остановку обучения в случае минимальных изменений от эпохи, а также за запись процесса обучения

In [ ]:
def compile_and_fit(model: tf.keras.Sequential, max_epochs: int = 50):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[
            'accuracy'
        ],
    )

    model.summary()

    history = model.fit(
        train_features, train_target,
        batch_size=BATCH_SIZE,
        validation_split=0.2,  # 20% of train data will validate model
        epochs=max_epochs,  # amount model trains
        verbose=2,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5),
            tf.keras.callbacks.TensorBoard(f'{LOGS_PATH}/{model.name}'),
        ],
    )

    return history.history

### Обучение моделей
Создадим переменную, в которой будут сохранятся логи обучения моделей

In [ ]:
size_histories = {}

#### Tiny
Начнём с самой маленькой модели, состоящей из двух обучающих слоёв

In [ ]:
tiny_model = tf.keras.Sequential(
    [
        normalize,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(train_target.shape[1]),
    ],
    name='tiny'
)

In [ ]:
size_histories['tiny'] = compile_and_fit(tiny_model)

Визуализируем ход обучения

In [ ]:
tiny_plot_history = sns.lineplot(data=pd.DataFrame(size_histories['tiny'])[['loss', 'val_loss']], linewidth=5)

tiny_plot_history.set_xlabel('Epoch')
tiny_plot_history.set_ylabel('Loss');

In [ ]:
tiny_plot_history.figure.savefig(f'{VISUAL_PATH}/fixed_tiny_history.png', transparent=True)

Посмотрим предсказательные возможности модели

In [ ]:
tiny_evaluation = tiny_model.evaluate(train_features, train_target, verbose=2)
print(f'Accuracy: {tiny_evaluation[1]}')

In [ ]:
tiny_evaluation = tiny_model.evaluate(test_features, test_target, verbose=2)
print(f'Accuracy: {tiny_evaluation[1]}')

Сохраним модель для возможности её использования в будущем

In [ ]:
tiny_model.save(f'{MODEL_PATH}/fixed_tiny')

#### Small

In [ ]:
small_model = tf.keras.Sequential(
    [
        normalize,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(train_target.shape[1]),
    ],
    name='small'
)

In [ ]:
size_histories['small'] = compile_and_fit(small_model)

Визуализируем ход обучения

In [ ]:
small_plot_history = sns.lineplot(data=pd.DataFrame(size_histories['small'])[['loss', 'val_loss']], linewidth=5)

small_plot_history.set_xlabel('Epoch')
small_plot_history.set_ylabel('Loss');

In [ ]:
small_plot_history.figure.savefig(f'{VISUAL_PATH}/fixed_small_history.png', transparent=True)

Посмотрим предсказательные возможности модели

In [ ]:
small_evaluation = small_model.evaluate(train_features, train_target, verbose=2)
print(f'Accuracy: {small_evaluation[1]}')

In [ ]:
small_evaluation = small_model.evaluate(test_features, test_target, verbose=2)
print(f'Accuracy: {small_evaluation[1]}')

Сохраним модель для возможности её использования в будущем

In [ ]:
small_model.save(f'{MODEL_PATH}/fixed_small')

#### Medium

In [ ]:
medium_model = tf.keras.Sequential(
    [
        normalize,
        tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dense(train_target.shape[1]),
    ],
    name='medium'
)

In [ ]:
size_histories['medium'] = compile_and_fit(medium_model)

Визуализируем ход обучения

In [ ]:
medium_plot_history = sns.lineplot(data=pd.DataFrame(size_histories['medium'])[['loss', 'val_loss']], linewidth=5)

medium_plot_history.set_xlabel('Epoch')
medium_plot_history.set_ylabel('Loss');

In [ ]:
medium_plot_history.figure.savefig(f'{VISUAL_PATH}/fixed_medium_history.png', transparent=True)

Посмотрим предсказательные возможности модели

In [ ]:
medium_evaluation = medium_model.evaluate(train_features, train_target, verbose=2)
print(f'Accuracy: {medium_evaluation[1]}')

In [ ]:
medium_evaluation = medium_model.evaluate(test_features, test_target, verbose=2)
print(f'Accuracy: {medium_evaluation[1]}')

Сохраним модель для возможности её использования в будущем

In [ ]:
medium_model.save(f'{MODEL_PATH}/fixed_medium')

#### Large

In [ ]:
large_model = tf.keras.Sequential(
    [
        normalize,
        tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.Dense(train_target.shape[1]),
    ],
    name='large'
)

In [ ]:
size_histories['large'] = compile_and_fit(large_model)

Визуализируем ход обучения

In [ ]:
large_plot_history = sns.lineplot(data=pd.DataFrame(size_histories['large'])[['loss', 'val_loss']], linewidth=5)

large_plot_history.set_xlabel('Epoch')
large_plot_history.set_ylabel('Loss');

In [ ]:
large_plot_history.figure.savefig(f'{VISUAL_PATH}/fixed_large_history.png', transparent=True)

Посмотрим предсказательные возможности модели

In [ ]:
large_evaluation = large_model.evaluate(train_features, train_target, verbose=2)
print(f'Accuracy: {large_evaluation[1]}')

In [ ]:
large_evaluation = large_model.evaluate(test_features, test_target, verbose=2)
print(f'Accuracy: {large_evaluation[1]}')

Сохраним модель для возможности её использования в будущем

In [ ]:
large_model.save(f'{MODEL_PATH}/fixed_large')

#### Итоговая модель

In [ ]:
fixed_plot_history = sns.lineplot(
    data=pd.concat(
        [pd.DataFrame(history)[['accuracy', 'val_accuracy']].rename(columns={'accuracy': f'{size}_accuracy', 'val_accuracy': f'{size}_val_accuracy'})
         for size, history in size_histories.items()],
        axis="columns"
    ),
    linewidth=5)

fixed_plot_history.set_xlabel('Epoch')
fixed_plot_history.set_ylabel('Loss');

In [ ]:
fixed_plot_history.figure.savefig(f'{VISUAL_PATH}/fixed_history.png', transparent=True)

Выберем в качестве основной модель с наименьшими ошибками

In [ ]:
fixed_model = large_model

### Диаграммы
Визуализируем полученные результаты - начнём с определения предсказанных значений

In [ ]:
test_predict = fixed_model.predict(test_features, verbose=2).flatten();

Визуализируем потери предсказаний в модели

In [ ]:
fixed_plot_relation = sns.regplot(x=test_target, y=test_predict)

fixed_plot_relation.set_xlabel('Target')
fixed_plot_relation.set_ylabel('Predict');

In [ ]:
fixed_plot_relation.figure.savefig(f'{VISUAL_PATH}/fixed_relation.png', transparent=True)

Визуализируем распределение ошибок модели

In [ ]:
fixed_plot_mistakes = sns.histplot(data=test_predict - test_target)

fixed_plot_mistakes.set_xlabel('Value');

Большинство наблюдений было предсказано правильно. При этом, учитывая правостороннюю ассиметрию, модель стремится определить большие показатели наблюдениям, чем они есть на самом деле.

In [ ]:
fixed_plot_mistakes.figure.savefig(f'{VISUAL_PATH}/fixed_mistakes.png', transparent=True)